In [ ]:
using DifferentialEquations
using Plots
gr(fmt="png")

In [ ]:
include("../src/HHModel.jl")

In [ ]:
?HHModel.ihcurrent

In [ ]:
ltk = HHModel.low_voltage_gated_potassium(15.0, subtype=:kv1);

# plot((x) -> HHModel.time_constant(ltk.m, x), -150, 20, legend=nothing)    
plot(ltk.m.infty, -150, 20, legend=nothing, ylim=(0, 1))

In [ ]:
hhna = HHModel.hh_sodium(20);

# plot(hhna.h.infty, -150, 20)
# plot!(hhna.m.infty, -150, 20)
plot((x) -> hhna.m.infty(x), -150, 20)